In [5]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

In [6]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

In [7]:
files = []
for file in os.listdir("coco/val2017"):
    if file.endswith(".jpg"):
        files.append(os.path.join("coco/val2017", file))

In [8]:
#Initialize the cuda timing
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [9]:
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval();

Downloading: "https://github.com/facebookresearch/detr/archive/master.zip" to C:\Users\Silvio/.cache\torch\hub\master.zip
Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to C:\Users\Silvio/.cache\torch\hub\checkpoints\detr-r50-e632da11.pth
100%|██████████| 159M/159M [00:14<00:00, 11.3MB/s]


In [22]:
time = []
for idx,path in enumerate(files):
    im = Image.open(path)
    transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    img = transform(im).unsqueeze(0)
     # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        img = img.to('cuda')
        model.to('cuda')

    # propagate through the model
    start.record()
    outputs = model(img)
    end.record()
    
    torch.cuda.synchronize()
    time.append(start.elapsed_time(end))
    print("Image {} has been analyzed".format(idx))
mean_time = sum(time)/len(time)
print("The mean time for the analysis of a image is {}".format(mean_time))

AttributeError: 'str' object has no attribute 'fromat'